# Pricing d'un call par perceptron multi-couches avec Numpy.




## 1 . Introduction

On s'intéresse à la mise en place intégrale d'un perceptron multi-couches pour établir le bon prix d'un call européen dans le monde de Black & Scholes. L'intérêt d'étudier un sous-jacent suivant une dynamique propre au modèle de Black & Scholes est de pouvoir créer une large base de données pour l'apprentissage, par exemple, et de pouvoir vérifier l'implémentation de notre modèle multi-couches. On se place, par la suite, dans l'espace de probabilité suivant $(\Omega, \mathcal{F}, \mathbb{Q}^{*})$, où $\mathbb{Q}^{*}$ désigne la probabilité risque-neutre.

On considère donc dans la suite, un sous-jacent $\{S_{t}\}_{t \geq 0}$ suivant la dynamique :
\begin{align}
    dS_{t} = rS_{t}dt + \sigma S_{t}dW_{t}
\end{align}
où r est le rendement constant et positif d'un actif sans risque, $\sigma$ la volatilité constante et positive du sous-jacent et $\{W_{t}\}_{t \geq 0}$ un mouvement brownien standard.

Par ailleurs, on pose $f$ la fonction $f : (t,x) \mapsto f(t,x) = ln(x)$. En appliquant la formule d'Itô à $f(t,S_t)$ puis en intégrant sur l'intervalle $[0, t]$ on a, d'après (1) :

\begin{align*}
    S_{t} = S_{0} e^{(r-\frac{1}{2}\sigma^{2})t + \sigma W_{t}}, \forall t \geq 0.
\end{align*}
On note alors que le processus actualisé $\{e^{-rt}S_{t}\}_{t\geq 0}$ est une martingale sous la probabilité risque neutre $\mathbb{Q}^{*}$.

## 2 . Call, payoff et prix dans le monde de Black & Scholes

### 2 . 1 . Rappels

On se donne une maturité $T \geq 0$, un strike $K > 0$ et on définit le payoff d'un call de la façon suivante :

\begin{align*}
    (S_T-K)_{+} = \max(S_T-K,0)
\end{align*}

Par définition, en notant $\mathcal{C}_{BS}$ le prix du call, on a :

\begin{align*}
    \mathcal{C}_{BS} = e^{-rT} \mathbb{E}_{\mathbb{Q}^{*}} [ (S_T-K)_{+} ]
\end{align*}

Lorsque l'actif $S$ a une dynamique suivant le modèle de Black-Scholes, on a une expression analytique de $\mathcal{C}_{BS}$:

\begin{align*}
    \mathcal{C}_{BS}& = S_0F(\alpha) - e^{-rT}KF(\beta),&\\
    \text{avec, } \alpha& = \frac{1}{\sigma \sqrt{T}}(\ln(\frac{S_0}{K})+(r+\frac{\sigma^{2}}{2})T)),&\\
    \text{et, }\beta& = \frac{1}{\sigma \sqrt{T}}(\ln(\frac{S_0}{K})+(r-\frac{\sigma^{2}}{2})T)).\\
\end{align*}
avec $F$ la fonction de répartition (CDF) d'une loi gaussienne centrée réduite.


$\underline{\textit{démonstration :}}$

On note que $(S_{T}-K)_{+} = (S_{T}-K)\mathbb{1}_{\{S_{T} \geq K \}}$ et que $\{S_{T} \geq K\} = \{ \sigma W_{T} \geq \ln(\frac{K}{S_0}) - (r - \frac{\sigma^{2}}{2})T\}$ d'après $(1)$. On sait aussi que $W_{T} \sim \mathcal{N}(0,T)$ par définition, donc finalement :
\begin{align*}
    W_{T} \overset{\mathcal{L}}{=} \sqrt{T} G, \quad G \sim \mathcal{N}(0,1) 
\end{align*}

On peut alors écrire $\{S_{T} \geq K \} = \{G \geq \frac{1}{\sigma\sqrt{T}}(\ln(\frac{K}{S_0}) - (r - \frac{\sigma^{2}}{2})T) \}$. Pour ne pas alourdir l'écriture, on note pour la suite $\gamma = \frac{1}{\sigma\sqrt{T}}(\ln(\frac{K}{S_0}) - (r - \frac{\sigma^{2}}{2})T)$.
Par ailleurs, par parité de la fonction de densité (PDF) de la loi gaussienne, il peut être nécessaire de rappeler que $G \overset{\mathcal{L}}{=} -G$.

On a alors, en développant nos calculs :

\begin{align*}
    \mathcal{C}_{BS}& = e^{-rT} \mathbb{E}_{\mathbb{Q}^{*}} [ (S_T-K) \mathbb{1}_{\{ G \geq \gamma \}}],&\\
    & = e^{-rT} \mathbb{E}_{\mathbb{Q}^{*}}[S_{T}\mathbb{1}_{\{ G \geq \gamma \}}] - e^{-rT} K \mathbb{E}_{\mathbb{Q}^{*}}[\mathbb{1}_{\{ G \geq \gamma \}}],&\\
\end{align*}

On commence par développer le terme $e^{-rT} K \mathbb{E}_{\mathbb{Q}^{*}}[\mathbb{1}_{\{ G \geq \gamma \}}]$:
\begin{align*}
    e^{-rT} K \mathbb{E}_{\mathbb{Q}^{*}}[\mathbb{1}_{\{ G \geq \gamma \}}] & = e^{-rT} K \mathbb{Q}^{*}(G \geq \gamma),&\\
    & = e^{-rT} K \mathbb{Q}^{*}(-G \geq \gamma),&\\
    & = e^{-rT} K F(\beta),&\\
\end{align*}

Développons ensuite $e^{-rT} \mathbb{E}_{\mathbb{Q}^{*}}[S_{T}\mathbb{1}_{\{ G \geq \gamma \}}]$:
\begin{align*}
    e^{-rT} \mathbb{E}_{\mathbb{Q}^{*}}[S_{T}\mathbb{1}_{\{ G \geq \gamma \}}] & = \frac{e^{-rT}}{\sqrt{2\pi}}\int_{\gamma}^{+\infty}S_{0}e^{(r-\frac{1}{2}\sigma^{2})T + \sigma \sqrt{T}x}e^{-\frac{x^2}{2}}dx,&\\
    & = \frac{S_0}{\sqrt{2\pi}}\int_{\gamma}^{+\infty}e^{-\frac{1}{2}\sigma^{2}T + \sigma \sqrt{T}x -\frac{x^2}{2}}dx,&\\
    & = \frac{S_0}{\sqrt{2\pi}}\int_{\gamma}^{+\infty}e^{-\frac{1}{2}(x-\sigma\sqrt{T})}dx,&\\
    & = \int_{\gamma - \sigma\sqrt{T}}^{+\infty}e^{-\frac{u^2}{2}}du, (u = x-\sigma\sqrt{T})&\\
    & = S_0 \mathbb{Q}^{*}(G \geq \gamma - \sigma\sqrt{T}),&\\
    & = S_0 \mathbb{Q}^{*}(-G \geq -\alpha), (-\alpha = \gamma - \sigma\sqrt{T})&\\
    & = S_0 F(\alpha).
\end{align*}

On a finalement par sommmation:
\begin{align*}
    \mathcal{C}_{BS} = S_0F(\alpha) - e^{-rT}KF(\beta).
\end{align*}

On peut alors contruire une fonction python qui calcule directement ce prix analytique en fonction des paramètres. On pourra par la suite utilisé cette fonction pour créer notre base de données d'apprentissage.

In [2]:
import numpy as np
import openturns as ot

In [3]:
def exactPriceCallBS(S0, r, sg, T, K):
    """
        Fonction calculant le prix d'un call européen dans le monde de B&S en utilisant l'expression analytique.
        -------
        Parameters:
    
    S0 = Spot du sous-jacent.
    r  = Rendement actif sans risque.
    sg = Volatilité du sous-jacent.
    T  = Maturité.
    K  = Strike.

        Returns:
    
    C  = Prix du call européen correspondant.
    """
    alpha = (np.log(S0/K) + (r + sg**2/2)*T)/(sg*np.sqrt(T))
    beta = (np.log(S0/K) + (r - sg**2/2)*T)/(sg*np.sqrt(T))
    N = ot.Normal()

    C = S0 * N.computeCDF(alpha) - np.exp(-r*T) * K * N.computeCDF(beta)  

    return C


### 2 . 2 . Données d'entraînement

Monte-Carlo? LHS?

## 3 . Perceptron multi-couches

Après avoir posé le contexte de notre étude, il s'agit ici d'expliciter le modèle neuronal permettant d'approcher $\mathcal{C}_{BS}$. On désire obtenir un modèle $f_{\omega}$, avec $\omega$ une paramétrisation associée, tel que $f_{\omega}(S_0,r,\sigma,T,K)$ soit une bonne approximation de $\mathcal{C}_{BS}(S_0,r,\sigma,T,K)$. La particularité d'avoir un modèle perceptron et qu'il va falloir l'entraîner pour avoir une paramétrisation $\hat{\omega}$ optimale. Pour ce faire, on dispose de valeurs de $\mathcal{C}_{BS}$ pour différentes combinaisons de $S_0,r,\sigma,T$ et $K$. La notion de bonne approximation sera détaillée par la suite mais elle est tout de même liée à la notion de performance. La performance de notre modèle va dépendre des hyperparamètres de notre perceptron, en particulier le nombre de sous-couches et le nombre de neurones par couches. Mais aussi des fonctions d'activation utilisées, elles seront indispensables lors de l'étape d'optimisation sous-jacente au problème d'apprentissage du modèle, ou encore des données d'entraînement. Ce que l'on veut à tout prix éviter c'est que notre modèle garde en mémoire les données d'entraînement, données auxquelles on connaît le prix $\mathcal{C}_{BS}$, à tel point qu'il ne peut plus s'en détacher.

### 3.1 . Structure du modèle

Dans les recherches de $\textit{Bennell}$ & $\textit{Sutcliffe}$ [1], un perceptron est utilisé pour estimer le prix d'un call européen. Entraîné à partir de données historiques, il est conçu pour ensuite être comparé au prix analytique d'un call d'après le modèle de Black & Scholes et ainsi mettre en lumière les limites de ce modèle. L'ambition est ici moindre dans le sens ou l'on souhaite construire un perceptron de manière à estimer le prix d'un call européen dans le monde de Black & Scholes. Il reste néanmoins intéressant de s'appuyer sur [1], pour avoir une idée de la structure du modèle. 

On fait le choix de construire dans un premier temps un perceptron à trois couches : une couche d'entrée, une couche intermédiaire et une couche de sortie. On pourra par la suite, mutliplier les couches intermédiaires si la performance de notre modèle n'est pas convaincante. Les travaux de $\textit{Goodfellow et al}$ [3] ont permis de mettre en lumière le fait que plus le réseau neuronal dispose de couches intermédiaires plus il est performant. 

La couche d'entrée est composée de cinq noeuds, correspondant aux paramètres du prix du call sous notre hypothèse faite sur la dynamique de l'actif $\{ S_{t}\}_{t\geq0}$, soit un noeud correspondant à $S_0$, à $r$, à $\sigma$, à $T$ et à $K$. On pourra s'intérroger, par la suite, quant à la nécessité d'adimensionner nos données d'entrées comme cela a été fait dans [1], toujours dans l'optique de rendre notre modèle plus performant.

Pour la couche intermédiaire, on fait le choix arbitraire de fixer sa contenance à dix noeuds. Comme pour le nombre de sous-couches, on pourra si nécessaire, ajouter plus de noeuds. On fait aussi le choix d'appliquer la même fonction d'activation pour tous les noeuds de la couche comme cela est courament fait dans la littérature. Pour la fonction d'activation, on choisit la fonction $\textit{softplus}$ pour la continuité de sa dérivée, en tout point, et la positivité de son support d'activation.

\begin{align*}
    f_{softplus} : x \mapsto f_{softplus}(x) = \ln(1 + e^{x})
\end{align*}
 
La couche de sortie est composée d'un seul noeud, contenant notre estimation $\hat{\mathcal{C}_{BS}}$. On applique aussi la fonction $\textit{softplus}$ à notre couche de sortie. Avant que notre modèle puisse estimer $\mathcal{C}_{BS}$ par une valeur non-aberante, il faut l'entrainer. C'est ce qui est détaillé dans la partie suivante. On conclut cette partie en ajoutant que chaque couche est reliée, aux noeuds, par des poids. Ces poids constituent la paramétrisation $\omega$ de notre modèle. Pour être complet sur la structure, il faudrait parler du taux d'apprentissage $\eta$ du réseau dont on discutera plus tard de la valeur qui lui est associé. On rappelle que l'on cherche $\hat{\omega}$, intéressons-nous alors à la paramétrisation du modèle. 

### 3.2 . Apprentissage du modèle

Notons $C$ la sortie de notre réseau non-entraîné. La phase d'apprentissage consiste à faire converger $C$ vers $\mathcal{C}_{BS}$ en fixant un critère d'arrêt. Ce critère peut-être un nombre d'itérations maximal, communément appelé $epoch$ dans la littérature, ou une tolérance fixé sur l'erreur d'apprentissage.

\begin{align*}
    C_{n} \underset{n \rightarrow N_{epoch}}{\rightarrow} \mathcal{C}_{BS}
\end{align*}

Pour quantifier l'erreur d'apprentissage, appelée par la suite erreur de propagation, il nous faut définir prélablement la fonction que l'on veut minimiser, à chaque itération $n$, avant de mettre à jour nos poids. Dans [1], c'est l'erreur moyenne quadratique qui est minimisée. On introduit alors la fonction suivante :

\begin{align*}
    \mathcal{J}(\omega) = \frac{1}{N_{batch}} \sum_{i = 1}^{N_{batch}} | C^{(i)} - \mathcal{C}_{BS} |^{2}
\end{align*}
où $N_{batch}$ est une partie des données dont on dispose pour entraîner notre perceptron. En particulier, il y autant de groupe de données de taille $N_{batch}$ qu'il y a d'iérations avant d'atteindre $N_{epoch}$. La minimisation de $\mathcal{J}$ doit donc permettre de trouver $\hat{\omega}$. On note que raisonner par groupe d'échantillons d'entrée de taille $N_{batch}$ accelère la convergence de l'apprentissage de notre réseau, par rapport à un apprentissage pour chaque jeu de données fourni en entrée. Pour expliquer concrètement comment apprend notre réseau, on introduit la notion d'algorithme de rétropropagation.

L'algorithme de rétropropagation permet de mettre à jour les paramètres $\Omega$ une fois que le réseau en phase d'apprentissage a pu fournir des résultats à partir d'entrées. Cette mise à jour doit permettre de minimiser la fonction $\mathcal{J}$ précédemment 
explicitée. Dans la suite, on note $e, 1, s$ les différentes couches de notre modèle et on introduit :

$\omega_{jk}^{l}$ : poids de la relation entre le k-ième neurone de la couche $l-1$ et le j-ième neurone de la couche l.

$a_{j}^{l}$ : valeur du j-ième neurone de la couche l.

$z_{j}^{l} = \sum_{k = 1}^{N_{l-1}} \omega_{jk}^{l}a_{k}^{l-1}$ avec $N_{l-1}$ le nombre de neurones sur la couche ${l-1}$.
    
On considère alors $\Omega = \{\omega_e, \omega_s\}$ pour où donc $\omega_e \in \mathcal{M}_{10,5}(\mathbb{R})$, $\omega_s \in \mathcal{M}_{1,10}(\mathbb{R})$. Pour faciliter l'écriture vectorielle du problème de propagation, on fait le choix de voir l'erreur de propagation entre une couche $l-1$ et $l$ comme une conséquence d'une perturbation sur la somme pondérée $z_{j}^{l} \forall j \in \mathbb{N}_{*}$ avec $j \leq N_{l}$.
On définit alors l'erreur $\epsilon_{j}^{l} = \frac{\partial \mathcal{J}}{\partial z_{j}^{l}}$. Il vient donc par dérivation composée que :
\begin{align*}
	\epsilon_{j}^{s} = \frac{\partial \mathcal{J}}{\partial a_{j}^{s}}\frac{\partial a_{j}^{s}}{\partial z_{j}^{s}} =  \frac{\partial \mathcal{J}}{\partial a_{j}^{s}} f_{softplus}^{'}(z_{j}^{s}), \quad j = 1 \text{ pour la dernière couche}.
\end{align*}

On note que comme on dispose que d'un noeud sur la dernière couche, $\epsilon_{1}^{s}$ est un scalaire qu'on notera $\epsilon^{s}$ dans la suite.

Cependant on cherche à trouver les paramètres du modèle permettant de minimiser $\mathcal{J}$, donc il faut pouvoir identifier $(\nabla_{\omega_{s}}\mathcal{J})_{1k} = \frac{\partial \mathcal{J}}{\partial \omega_{1k}^{s}}$. En fait, on a la relation de fermeture évidente : 
\begin{align*}
	\frac{\partial \mathcal{J}}{\partial \omega_{1k}^{s}} = \frac{\partial \mathcal{J}}{\partial z_{1}^{s}} \frac{\partial z_{1}^{s} }{\partial \omega_{1k}^{s}}  = \epsilon^{s}a_{k}^{1}.
\end{align*}
On note qu’on est autorisé à chercher une telle matrice $\omega^{s}$ puisque $\mathcal{J}$ est quadratique. La mise à jour des poids requiert donc un algorithme d'optimisation.

Le choix d'optimisation étant à ce stade relativement libre, on se tourne vers l'algorithme d'Adam, détaillé dans les travaux de $\textit{Kingma}$ & $\textit{Ba}$ [2], pour sa facilité d'implémentation, son faible coût mémoire et sa performance.

\begin{align*}
    \omega_{s} = \omega_{s} - \eta \frac{\hat{M}}{\sqrt{\hat{V}} + \alpha}
\end{align*}
où $\hat{M}$ et $\hat{V}$ sont respectivement un estimateur de la moyenne et un estimateur de la variance du gradient de la fonction coût à minimiser et $\alpha$ est un coefficient arbitraire permettant
de se prémunir des problèmes éventuels de la division. Dans le cas ci-dessus, $\eta$ est en fait le pas lors de la descente de gradient, d'où son nom de taux d'apprentissage. Bien évidemment un grand pas permet de converger rapidement mais avec une paramétrisation potentiellement
peu satisfaisante du modèle neuronal, tandis qu'un pas faible permet d'avoir une paramétrisation satisfaisante tout en ayant un temps d'apprentissage plus conséquent.

L'algorithme d'Adam permet de mettre à jour les poids $\omega_s$. Il faut donc aussi propager cette erreur pour mettre à jour les autres paramètres du modèle. Mais donc comme les neurones sont liés, d'une couche à l'autre par les poids et la fonctions d'activation 
il est possible de propager l'erreur obtenue. 
Autrement dit, on peut facilement obtenir $\nabla_{\omega_{e}}\mathcal{J}$ à partir de $\nabla_{\omega_{s}}\mathcal{J}$. En effet, soit $j \in \mathbb{N}$, $j \leq 10$ fixé, puisqu'il y a dix noeuds sur la couche intermédiaire, on a alors :
\begin{align*}
    \epsilon_{j}^{1} &= \frac{\partial \mathcal{J}}{\partial z_{j}^{1}},&\\
     &= \frac{\partial \mathcal{J}}{\partial a_{j}^{1}} f_{softplus}^{‘}(z_{j}^{1}),\\
     &= \frac{\partial \mathcal{J}}{\partial a_{1}^{s}}  \frac{\partial a_{1}^{s}}{\partial a_{j}^{1}} f_{softplus}^{‘}(z_{j}^{1}),&\\
     &= \omega_{1j}^{s} \epsilon^{s} f_{softplus}^{‘}(z_{j}^{1}),&\\
\end{align*}

Ce qui donne sous forme vectorielle : 
\begin{align*}
	\epsilon^{1} = \epsilon^{s} (\omega^{s})^{T} \star f_{softplus}^{‘}(z^{1})
\end{align*}


### 3.3 . Implémentation du modèle avec Numpy

On note qu'une approche objet serait plus adaptée pour ce projet ne serait-ce que pour pouvoir adapter plus facilement notre modèle, mais pour simplifier l'implémentation, on se restreint ici à une implémentation directe.

In [1]:
########## Structure du perceptron

# Hyperparamètres de structure et fonction(s) d'activation

Ne = 5 #noeuds en entrée
Ni = 10 #noeuds intermédiaires
Ns = 1 #noeuds en sortie

f_soft = lambda x : np.log(1+np.exp(x)) 
fp_soft = lambda x : 1/(1+np.exp(-x)) 

# Structure

def MLP(Input, We, Ws, f_a1, f_a2):
    """
        Fonction implémentant le modèle neuronale avec 3 couches.
        --------
        Parameters:
    
    Input : entrée du modèle.
    We    : poids entre la couche d'entrée et la couche intermédiaire.
    Ws    : poids entre la couche intermédiaire et la couche de sortie.
    f_a1  : fonction d'activation pour la couche intermédiaire.
    f_a2  : fonction d'activation pour la couche de sortie.

        Returns:
    
    Output : sortie du modèle.
    
    """
    # Passage couche d'entrée à couche intermédiaire
    Inter = np.dot(We,Input)

    # Activation couche intermédiaire
    InterAct = f_a1(Inter)

    # Passage couche intermédiaire à couche de sortie
    Output = np.dot(Ws, InterAct)

    # Activation couche de sortie
    OutputAct = f_a2(Output)

    return OutputAct, Output, InterAct, Inter


In [3]:
########## Apprentissage du perceptron

# Hyperparamètres d'apprentissage

Nepochs = 1000  #nombre d'epochs
eta = 0.001     #taux d'apprentissage
tol = 10e-6     #tolérance MSE
Nbatch = 32    

# Apprentissage

def PropagationMPL(InputNbatch, We, Ws, f_a1, f_a2):
    """
        Fonction de propagation.
        --------
        Parameters:

    InputNbatch : groupe d'entrées + 1 sortie de référence pour chacune des entrées (nécessaire pour MSE_Nbatch). Taille : (6,Nbatch)
    We          : poids entre la couche d'entrée et la couche intermédiaire.
    Ws          : poids entre la couche intermédiaire et la couche de sortie.
    f_a1        : fonction d'activation pour la couche intermédiaire.
    f_a2        : fonction d'activation pour la couche de sortie.

        Returns:
    
    OutputNbatch : groupe de sorties. Taille : (1,Nbatch)
    MSE_Nbatch   : Erreur quadratique sur l'ensemble des sorties.
    """
    # Initialisation
    Nbatch = np.shape(InputNbatch)[1] #entrées stockées par colonne

    OutputNbatch = np.zeros(Nbatch)

    # Séparation entrées/sorties de référence
    OutputNbatchRef = InputNbatch[5,:]
    InputNbatch = InputNbatch[0:5,:]

    # Propagation
    for i in range(Nbatch):
        As, Zs, A1, Z1 = MLP(InputNbatch[:,i], We, Ws, f_a1, f_a2)
        OutputNbatch[i] = As
        
    # Calcul MSE
    MSE_Nbatch = np.sum(abs(OutputNbatch-OutputNbatchRef)**2)/Nbatch

    ME_Nbatch = np.sum(abs(OutputNbatch-OutputNbatchRef))/Nbatch            

    return OutputNbatch, MSE_Nbatch, ME_Nbatch, As, Zs, A1, Z1

def AdamOpt(eta, gradJ, W, M, V, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8):
    """
        Fonction effectuant 1 itération de l'algorithme d'Adam pour ajuster les poids selon la descente de gradient.
        --------
        Parameters:

    eta     : taux d'apprentissage.
    gradJ   : gradient fonction coût par rapport à W.
    W       : poids de la couche.
    M       : estimation de la moyenne.
    V       : estimation de la variance.
    beta1   : paramètre 1 de l'algorithme d'Adam.
    beta2   : paramètre 2 de l'algorithme d'Adam.
    epsilon : paramètre préventif pour la division.

        Returns:
    
    Wmaj    : poids mis à jour.
    Mmaj    : estimation de la moyenne mise à jour.
    Vmaj    : estimation de la variance mise à jour.
    """
    # Calculs préliminaires
    Mmaj = beta1*M + (1-beta1)*gradJ
    Vmaj = beta2*V + (1-beta2)*(gradJ**2)

    Mmaj = Mmaj/(1-beta1)
    Vmaj = Vmaj/(1-beta2)

    # Mise à jour
    Wmaj = W - eta * Mmaj/(np.sqrt(Vmaj) + epsilon)

    return Wmaj, Mmaj, Vmaj

def ComputeGradient(ME_Nbatch, Zs, A1, Z1, InputNbatch, Ws, fp_a1, fp_a2):
    """
        Fonction permettant de calculer les gradients à partir des données de la dernière itération de la 
        propagation par paquet de taille Nbatch.
        --------
        Parameters:
    
    ME_Nbatch : Moyenne des écarts, en valeur absolue, aux données de référence du paquet de taille Nbatch.
    Zs        : Valeur du neurone de sortie avant activation.
    A1        : Valeurs finales des neurones de la couche intermédiaire.
    Z1        : Valeurs avant activation des neurones de la couche intermédiaire.
    InputNbatch : Valeurs des entrées à la dernière itération de propagation des données.
    Ws        : Poids entre le neurone de sortie et la couche intermédiaire.
    fp_a1     : Dérivée de la fonction d'activation pour la couche intermédiaire.
    fp_a2     : Dérivée de la fonction d'activation pour la couche (1 neurone) de sortie.

        Returns:

    gradJe    : Matrice gradient de J par rapport à chacun des poids We. Taille : (10, 5)
    gradJs    : Vecteur gradient de J par rapport à chacun des poids Ws. Taille : (1, 10)
    """
    # Compute gradJs
    epsilon_s = 2*ME_Nbatch * fp_a2(Zs) #scalaire puisque Zs est scalaire
    gradJs = epsilon_s*A1

    # Commpute gradJe
    epsilon_1 = epsilon_s* Ws * fp_a1(Z1)
    gradJe = np.outer(epsilon_1, InputNbatch)

    return gradJe, gradJs

def RetropropagationMLP(eta, gradJe, gradJs, We, Me, Ve, Ws, Ms, Vs):
    """
        Fonction permettant de rétropropager l'erreur dans le réseau.
        --------
        Parameters:
    
    eta     : taux d'apprentissage.
    gradJe  : gradient fonction coût par rapport à We.
    gradJs  : gradient fonction coût par rapport à Ws.
    We      : poids couche d'entrée.
    Me      : estimation de la moyenne.
    Ve      : estimation de la variance.
    Ws      : poids couche d'entrée.
    Ms      : estimation de la moyenne.
    Vs      : estimation de la variance.

        Returns:
    
    We_maj    : poids We mis à jour.
    Me_maj    : estimation de la moyenne mise à jour.
    Ve_maj    : estimation de la variance mise à jour.
    Ws_maj    : poids Ws mis à jour.
    Ms_maj    : estimation de la moyenne mise à jour.
    Vs_maj    : estimation de la variance mise à jour.
    """
    # We
    We_maj, Me_maj, Ve_maj = AdamOpt(eta, gradJe, We, Me, Ve, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8)

    # Ws
    Ws_maj, Ms_maj, Vs_maj = AdamOpt(eta, gradJs, Ws, Ms, Vs, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8)

    return We_maj, Me_maj, Ve_maj, Ws_maj, Ms_maj, Vs_maj

def Apprentissage(InputApp, Nepochs, Nbatch, f_a1, f_a2, fp_a1, fp_a2, eta, tol):
    """
        Fonction réalisant l'apprentissage du modèle.
        --------
        Parameters:

    InputApp : données d'apprentissage. Taille : (6,Nepochs*Nbatch) à minima voir plus si cross-validation.
    Nepochs  : nombre d'itérations pour l'apprentissage. 
    Nbatch   : nombre d'entrées pour la mise à jour/rétropropagation.
    eta      : taux d'apprentissage.
    f_a1     : fonction d'activation pour la couche intermédiaire.
    f_a2     : fonction d'activation pour la couche de sortie.
    fp_a1    : dérivée de la fonction d'activation pour la couche intermédiaire.
    fp_a2    : dérivée de la fonction d'activation pour la couche (1 neurone) de sortie.
    tol      : tolérance sur la valeur de J après propagation.

        Returns:

    WeOpt    : poids optimaux pour la couche We.
    WsOpt    : poids optimaux pour la couche Ws.
    """
    # for i in range(Nepochs)
    #   change les entrées de la propagation
    #   Propagation
    #   Gradient de J
    #   Si MSE_Nbatch > tol :
    #     Retropropagation -> récuperer les W, M, V pour chaque couche

    # Initialisation des poids 
    Ws = np.zeros((Ns,Ni))
    We = np.zeros((Ni,Ne))

    # Initialisation des moments
    Me = np.zeros(np.shape(We))
    Ve = np.zeros(np.shape(We))

    Ms = np.zeros(np.shape(Ws))
    Vs = np.zeros(np.shape(Ws))
    
    # Apprentissage
    for i in range(Nepochs):
        print(str(i)+' epochs')
        InputNbatch = InputApp[i*Nbatch,(i+1)*Nbatch]
        OutputNbatch, MSE_Nbatch, ME_Nbatch, As, Zs, A1, Z1 = PropagationMPL(InputNbatch, We, Ws, f_a1, f_a2)

        if(MSE_Nbatch > tol):
            gradJe, gradJs =  ComputeGradient(ME_Nbatch, Zs, A1, Z1, InputNbatch[:,Nbatch], fp_a1, fp_a2)
            We, Me, Ve, Ws, Ms, Vs = RetropropagationMLP(eta, gradJe, gradJs, We, Me, Ve, Ws, Ms, Vs)

    WeOpt = We
    WsOpt = Ws

    return WeOpt, WsOpt

## Référence

[1] - Bennell, Julia & Sutcliffe Charles, $\textit{Black-Scholes versus artificial neural networks in pricing FTSE 100 options}$, online, Wiley InterScience, 2004.

[2] - Kingma, Diederik P. & Ba, Jimmy Lei, $\textit{Adam: a method for stochastic optimization}$, conference paper, ICLR, 2015.

[3] - Goodfellow, Ian & Bengio, Yoshua & Courville, Aaron, $\textit{Deep learning}$, published, MIT Press, 2016.

[4] - Nielsen, Michael, $\textit{Neural Networks and Deep learning}$, online, Determination Press, 2015.

[5] - Nsenge, Mpia Héritier & Inipaivudu, Baelani Nephtali, $\textit{L’Algorithme de rétro-propagation de gradient dans le perceptron multicouche: Bases et étude de cas}$, online, Innovative Space of Scientific Research Journals, 2021.